In [7]:
def load_results(source_dir, dates):
    df_results = pd.DataFrame()
    for date in dates:
        df = open_file(source_dir + '/Suggested_links_' + date + '.csv')
        df['datetime'] = date
        df_results = pd.concat([df_results, df], ignore_index=True, sort=False)
    
    df_results.drop_duplicates()
    return df_results

In [8]:
def links_per_day(df):
    df_total_links = df[['primary_gnb', 'selected', 'datetime']]
    df_total_links = (
        df_total_links[['primary_gnb', 'selected', 'datetime']]
        .groupby(['selected', 'datetime'])
        .agg({'selected': 'count'})
    )

    df_total_links.rename({'selected': 'total'}, axis=1, inplace=True)
    df_total_links.reset_index(inplace=True)

    df_total_links = pd.pivot_table(
        df_total_links, 
        values="total",
        index="selected",
        columns="datetime", 
        aggfunc=np.mean
    )

    return df_total_links

In [9]:
def links_distribution(df):
    df_total_links = df[['primary_gnb', 'selected', 'datetime']]
    df_total_links = (
        df_total_links[['primary_gnb', 'selected', 'datetime']]
        .groupby(['primary_gnb', 'datetime'])
        .agg({'selected': 'count'})
    )
    df_total_links.reset_index(inplace=True)

    df_total_links = (
        df_total_links[['primary_gnb', 'selected', 'datetime']]
        .groupby(['selected', 'datetime'])
        .agg({'primary_gnb': 'count'})
    )
    df_total_links.reset_index(inplace=True)

    df_total_links = pd.pivot_table(
        df_total_links, 
        values="primary_gnb",
        index="selected",
        columns="datetime", 
        aggfunc=np.mean
    )

    return df_total_links

In [10]:
def links_distribution_nodes(df):
    df_suggested_links = df.query('selected == True')[['primary_gnb', 'selected', 'datetime']]
    df_suggested_links = (
        df_suggested_links[['primary_gnb', 'selected', 'datetime']]
        .groupby(['primary_gnb', 'datetime'])
        .agg({'selected': 'count'})
    )
    df_suggested_links.reset_index(inplace=True)

    df_suggested_links = (
        df_suggested_links[['primary_gnb', 'selected', 'datetime']]
        .groupby(['selected', 'datetime'])
        .agg({'primary_gnb': 'count'})
    )
    df_suggested_links.reset_index(inplace=True)

    df_suggested_links = pd.pivot_table(
        df_suggested_links, 
        values="primary_gnb",
        index="selected",
        columns="datetime", 
        aggfunc=np.mean
    )

    return df_suggested_links

In [11]:
def get_rate_of_change(df1, df2):
    df_delta = df1.merge(df2, on=['gNbs'], how='left', indicator=True)
    df_delta = df_delta._merge.value_counts()
    df_delta.rename({'both': 'common','left_only': 'removed',
                          'right_only': 'new'}, inplace=True)
    df_delta['total'] = df2.gNbs.count()
    df_delta['new'] = df2.gNbs.count()-df_delta['common']
    return df_delta


def roc_over_days(df, dates):
    df_roc = pd.DataFrame()
    df_16 = df[(df['datetime'] == '2022-08-16') & df['selected'] == True]
    df_17 = df[(df['datetime'] == '2022-08-17') & df['selected'] == True]
    df_18 = df[(df['datetime'] == '2022-08-18') & df['selected'] == True]
    df_19 = df[(df['datetime'] == '2022-08-19') & df['selected'] == True]
    df_20 = df[(df['datetime'] == '2022-08-20') & df['selected'] == True]
    df_21 = df[(df['datetime'] == '2022-08-21') & df['selected'] == True]

    df_roc['2022-08-17'] = get_rate_of_change(df_16, df_17)
    df_roc['2022-08-18'] = get_rate_of_change(df_17, df_18)
    df_roc['2022-08-19'] = get_rate_of_change(df_18, df_19)
    df_roc['2022-08-20'] = get_rate_of_change(df_19, df_20)
    df_roc['2022-08-21'] = get_rate_of_change(df_20, df_21)
    df_roc['measure'] = df_roc.index

    df_roc = pd.melt(df_roc, id_vars='measure', value_vars=dates)
    df_roc = pd.pivot_table(
        df_roc, 
        values='value',
        index='variable',
        columns='measure', 
        aggfunc=np.mean
    )

    df_roc['common_percent'] = (df_roc['common'] * 100) / df_roc['total']
    df_roc['new_percent'] = (df_roc['new'] * 100) / df_roc['total']
    df_roc['removed_percent'] = (df_roc['removed'] * 100) / df_roc['total']

    return df_roc

In [12]:
def hitrate_coverage(df):

    # (sum of True links) / (sum of all links)

    df_total_hitrate = (
        df[['primary_gnb', 'hitrate', 'datetime']]
        .groupby(['primary_gnb', 'datetime'])
        .agg({'hitrate': 'sum'})
    )
    df_total_hitrate.reset_index(inplace=True)
    df_total_hitrate.rename({'hitrate': 'hitrate_total'}, axis=1, inplace=True)


    df_created_links = df.query('selected == True')[['primary_gnb', 'datetime', 'hitrate']]
    df_created_links = (
        df_created_links[['primary_gnb', 'hitrate', 'datetime']]
        .groupby(['primary_gnb', 'datetime'])
        .agg({'hitrate': 'sum'})
    )

    df_hitrate = df_total_hitrate.merge(df_created_links, right_on=['primary_gnb', 'datetime'], left_on=['primary_gnb', 'datetime'])

    df_hitrate['hitrate'] = (df_hitrate['hitrate'] * 100) / df_hitrate['hitrate_total']
    df_hitrate['hitrate'] = df_hitrate['hitrate'].round(decimals=0)

    df_hitrate = (
        df_hitrate[['primary_gnb', 'hitrate', 'datetime']]
        .groupby(['hitrate', 'datetime'])
        .agg({'primary_gnb': 'count'})
    )
    df_hitrate.reset_index(inplace=True)

    df_hitrate = pd.pivot_table(
        df_hitrate, 
        values="primary_gnb",
        index="hitrate",
        columns="datetime", 
        aggfunc=np.mean
    )

    return df_hitrate

In [ ]:
def get_removed_link_counts(df1, df2):
    df_delta = df1.merge(df2, on=['gNbs'], how='left', indicator=True)
    df_delta = df_delta[df_delta['_merge'] == 'left_only']
    return df1[ (df1['gNbs'].isin(df_delta['gNbs'])) ]

def removed_links(df):
    df_16 = df_results[(df_results['datetime'] == '2022-08-16') & df_results['selected'] == True]
    df_17 = df_results[(df_results['datetime'] == '2022-08-17') & df_results['selected'] == True]
    df_18 = df_results[(df_results['datetime'] == '2022-08-18') & df_results['selected'] == True]
    df_19 = df_results[(df_results['datetime'] == '2022-08-19') & df_results['selected'] == True]
    df_20 = df_results[(df_results['datetime'] == '2022-08-20') & df_results['selected'] == True]
    df_21 = df_results[(df_results['datetime'] == '2022-08-21') & df_results['selected'] == True]

    df_roc_analysis = pd.DataFrame()

    df_roc_analysis = pd.concat([df_roc_analysis, get_removed_link_counts(df_16, df_17)], ignore_index=True, sort=False)
    df_roc_analysis = pd.concat([df_roc_analysis, get_removed_link_counts(df_17, df_18)], ignore_index=True, sort=False)
    df_roc_analysis = pd.concat([df_roc_analysis, get_removed_link_counts(df_18, df_19)], ignore_index=True, sort=False)
    df_roc_analysis = pd.concat([df_roc_analysis, get_removed_link_counts(df_19, df_20)], ignore_index=True, sort=False)
    df_roc_analysis = pd.concat([df_roc_analysis, get_removed_link_counts(df_20, df_21)], ignore_index=True, sort=False)

    df_roc_analysis = (
        df_roc_analysis[['primary_gnb', 'selected', 'datetime']]
        .groupby(['primary_gnb', 'datetime'])
        .agg({'selected': 'count'})
    )
    df_roc_analysis.reset_index(inplace=True)
    df_removed = df_roc_analysis.copy()

    df_roc_analysis = (
        df_roc_analysis[['primary_gnb', 'selected', 'datetime']]
        .groupby(['selected', 'datetime'])
        .agg({'primary_gnb': 'count'})
    )
    df_roc_analysis.reset_index(inplace=True)

    df_roc_analysis = pd.pivot_table(
        df_roc_analysis, 
        values="primary_gnb",
        index="selected",
        columns="datetime", 
        aggfunc=np.mean
    )

    return df_roc_analysis

In [ ]:
def apply_rank(primary_gnb, datetime, counts):
    id = str(primary_gnb) + ':' + datetime
    if id not in counts:
        counts[id] = 0
    
    counts[id]+=1
    return counts[id]

def rank_links(df):
    counts = {}
    df_sorted_results = df.sort_values(
            ['datetime', 'secondary_gnb', 'usability'], ascending=False
        )

    df_sorted_results['secondary_rank'] = df_sorted_results.apply(lambda x: apply_rank(x.secondary_gnb, x.datetime, counts), axis=1)

    counts = {}
    df_sorted_results = df_sorted_results.sort_values(
            ['datetime', 'primary_gnb', 'usability'], ascending=False
        )

    df_sorted_results['primary_rank'] = df_sorted_results.apply(lambda x: apply_rank(x.primary_gnb, x.datetime, counts), axis=1)

    df_sorted_results.reset_index(drop=True, inplace=True)

    return df_sorted_results

In [ ]:
#https://robert-haas.github.io/gravis-docs/code/examples/external_tools/networkx.html

def get_node_colour(node_id):
    if node_id == nodename:
        return 'blue'
    else:
        return 'orange'
    
def get_edge_details(dataframe, edge_details):
    source = edge_details[0]
    target = edge_details[1]
    datarow = dataframe.loc[ (dataframe['primary_node_name'] == source) & (dataframe['secondary_node_name'] == target) ]
    usability = str( datarow['usability'].iloc[0] )
    
    if str( datarow['selected'].iloc[0] ) == 'False':
        colour = 'grey'
    elif str( datarow['unwanted'].iloc[0] ) == 'True':
        colour = 'red'
    elif str( datarow['mandatory'].iloc[0] ) == 'True':
        colour = 'blue'
    else:
        colour = 'green'
    return usability, colour

def render_network_graph(dataframe):
    graph = nx.from_pandas_edgelist(dataframe,source='primary_node_name',
                                   target='secondary_node_name',edge_attr=None,
                                   create_using=nx.DiGraph())
    for node_id in graph.nodes:
        node = graph.nodes[node_id]
        node['color'] = get_node_colour(node_id)
        node['fontsize'] = 9
    
    for edge_id in graph.edges:
        usability, colour = get_edge_details(dataframe, edge_id)
        edge =  graph.edges[edge_id]
        edge['color'] = colour
        edge['size'] = 2
        edge['labels'] = usability
    return graph